# Vietnamese-English Translation với PyTorch 2.5.1

**Hướng dẫn đơn giản để fine-tune model dịch Việt-Anh**

## 🎯 Mục tiêu
- Fine-tune model mT5 cho dịch thuật Việt-Anh
- Sử dụng LoRA để training hiệu quả
- Tương thích hoàn toàn với PyTorch 2.5.1+cu121

## 📋 Các bước thực hiện
1. ✅ Cài đặt thư viện cần thiết
2. ✅ Tạo dataset Vietnamese-English
3. ✅ Load và chuẩn bị model mT5
4. ✅ Setup LoRA cho efficient training
5. ✅ Training với parameters tối ưu
6. ✅ Test và đánh giá kết quả

## 🔧 Yêu cầu hệ thống
- Python 3.8+
- PyTorch 2.5.1+cu121
- GPU với CUDA 12.1 (hoặc CPU)
- RAM: 8GB+ (16GB khuyến nghị)

## Bước 1: Cài đặt thư viện

**Quan trọng:** Sử dụng PyTorch 2.5.1+cu121 để tương thích tốt nhất

In [ ]:
# Cài đặt PyTorch 2.5.1 với CUDA 12.1
!pip install torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121

# Cài đặt thư viện machine learning
!pip install transformers>=4.44.0 datasets>=2.20.0 accelerate>=0.33.0 peft>=0.12.0 safetensors>=0.4.0

# Cài đặt thư viện đánh giá
!pip install evaluate rouge-score

# Cài đặt thư viện phụ trợ
!pip install pandas numpy matplotlib tqdm

print("✅ Đã cài đặt tất cả thư viện cần thiết!")

In [ ]:
# Import thư viện cần thiết
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset, DatasetDict
import evaluate

# PEFT cho LoRA
from peft import LoraConfig, get_peft_model, TaskType

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Sử dụng device: {device}")
print(f"🐍 Python version: {torch.__version__}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
else:
    print("⚠️ Chạy trên CPU - sẽ chậm hơn")

# Đặt random seeds
torch.manual_seed(42)
np.random.seed(42)
print("✅ Setup hoàn tất!")

## Bước 2: Tạo Dataset Vietnamese-English

Tạo dataset đơn giản để demo. Trong thực tế, bạn sẽ sử dụng dataset lớn hơn.

In [ ]:
# Tạo dataset Vietnamese-English đơn giản, dataaset sẽ trông như thế này:
data = [
    {"vn": "Xin chào", "en": "Hello"},
    {"vn": "Tạm biệt", "en": "Goodbye"},
    {"vn": "Cảm ơn bạn", "en": "Thank you"},
    {"vn": "Xin lỗi", "en": "Sorry"},
    {"vn": "Bạn khỏe không?", "en": "How are you?"},
    {"vn": "Tôi tên là Nam", "en": "My name is Nam"},
    {"vn": "Hôm nay thời tiết đẹp", "en": "The weather is nice today"},
    {"vn": "Tôi thích ăn phở", "en": "I like to eat pho"},
    {"vn": "Chúc bạn ngủ ngon", "en": "Good night"},
    {"vn": "Hẹn gặp lại", "en": "See you later"},
    {"vn": "Tôi đang học tiếng Anh", "en": "I am learning English"},
    {"vn": "Bạn có thể giúp tôi không?", "en": "Can you help me?"},
    {"vn": "Tôi yêu Việt Nam", "en": "I love Vietnam"},
    {"vn": "Hà Nội là thủ đô", "en": "Hanoi is the capital"},
    {"vn": "Món này rất ngon", "en": "This dish is delicious"},
    {"vn": "Tôi muốn đi du lịch", "en": "I want to travel"},
    {"vn": "Gia đình tôi có 4 người", "en": "My family has 4 people"},
    {"vn": "Tôi làm việc ở văn phòng", "en": "I work at the office"},
    {"vn": "Chúng ta đi ăn nhé", "en": "Let's go eat"},
    {"vn": "Tôi thích xem phim", "en": "I like watching movies"}
]

df = pd.DataFrame(data)
print(f"📊 Tạo dataset với {len(df)} cặp câu")
print("\n🔍 Một số ví dụ:")
for i in range(3):
    print(f"🇻🇳 {df.iloc[i]['vn']} → 🇺🇸 {df.iloc[i]['en']}")
    
print(f"\n📈 Thống kê:")
print(f"Trung bình từ tiếng Việt: {df['vn'].str.split().str.len().mean():.1f}")
print(f"Trung bình từ tiếng Anh: {df['en'].str.split().str.len().mean():.1f}")

In [ ]:
# Chuẩn bị data cho T5 format
def create_t5_format(vn_text, en_text):
    """Tạo format cho T5: 'translate Vietnamese to English: [VN]' -> '[EN]'"""
    input_text = f"translate Vietnamese to English: {vn_text}"
    target_text = en_text
    return input_text, target_text

# Tạo dataset với T5 format
inputs = []
targets = []

for _, row in df.iterrows():
    inp, tgt = create_t5_format(row['vn'], row['en'])
    inputs.append(inp)
    targets.append(tgt)

print("🔄 Ví dụ T5 format:")
print(f"Input: {inputs[0]}")
print(f"Target: {targets[0]}")

# Tạo Hugging Face Dataset
dataset = Dataset.from_dict({
    'input_text': inputs,
    'target_text': targets
})

# Chia train/validation (80/20)
dataset = dataset.shuffle(seed=42)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, len(dataset)))

print(f"\n📊 Chia dataset:")
print(f"Training: {len(train_dataset)} samples")
print(f"Validation: {len(val_dataset)} samples")

## Bước 3: Load Model và Tokenizer

Sử dụng mT5-small - model nhỏ nhưng hiệu quả cho demo

In [ ]:
# Load model và tokenizer
MODEL_NAME = "google/mt5-small"
print(f"📥 Loading model: {MODEL_NAME}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"📚 Tokenizer vocabulary: {len(tokenizer):,} tokens")

# Test tokenization
test_text = "translate Vietnamese to English: Xin chào"
tokens = tokenizer.tokenize(test_text)
print(f"\n🧪 Test tokenization:")
print(f"Text: {test_text}")
print(f"Tokens: {tokens[:10]}...")  # Chỉ hiển thị 10 tokens đầu
print(f"Token IDs: {tokenizer.encode(test_text)[:10]}...")

print("✅ Tokenizer sẵn sàng!")

In [ ]:
# Prepare data in T5 format: "translate Vietnamese to English: [VN_TEXT]" -> "[EN_TEXT]"
def create_t5_format(vietnamese_text, english_text):
    """Create T5 input-output format."""
    input_text = f"translate Vietnamese to English: {vietnamese_text}"
    target_text = english_text
    return input_text, target_text

# Create T5 formatted data
inputs = []
targets = []

for _, row in df.iterrows():
    input_text, target_text = create_t5_format(
        row['vietnamese_clean'], 
        row['english_clean']
    )
    inputs.append(input_text)
    targets.append(target_text)

# Create Hugging Face Dataset
dataset_dict = {
    'input_text': inputs,
    'target_text': targets,
    'vietnamese': df['vietnamese_clean'].tolist(),
    'english': df['english_clean'].tolist()
}

dataset = Dataset.from_dict(dataset_dict)

# Split into train/validation sets (80/20 split)
train_size = int(0.8 * len(dataset))
dataset = dataset.shuffle(seed=42)
train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, len(dataset)))

dataset_splits = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

print(f"📊 Dataset splits:")
print(f"  Training: {len(train_dataset)} examples")
print(f"  Validation: {len(val_dataset)} examples")

# Show example of formatted data
print(f"\n🔍 Example formatted data:")
example = train_dataset[0]
print(f"Input: {example['input_text']}")
print(f"Target: {example['target_text']}")

# Tokenize dataset
max_length = 128

def tokenize_function(examples):
    """Tokenize input và target texts"""
    # Tokenize inputs
    inputs = tokenizer(
        examples['input_text'],
        max_length=max_length,
        truncation=True,
        padding=False  # Padding động trong training
    )
    
    # Tokenize targets
    targets = tokenizer(
        examples['target_text'],
        max_length=max_length,
        truncation=True,
        padding=False
    )
    
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize cả train và validation
print("🔄 Tokenizing dataset...")
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Xóa columns không cần thiết
train_dataset = train_dataset.remove_columns(['input_text', 'target_text'])
val_dataset = val_dataset.remove_columns(['input_text', 'target_text'])

print(f"✅ Tokenization hoàn tất!")
print(f"📊 Train dataset: {len(train_dataset)} samples")
print(f"📊 Val dataset: {len(val_dataset)} samples")

# Kiểm tra 1 sample
sample = train_dataset[0]
print(f"\n🔍 Sample tokenized:")
print(f"Input length: {len(sample['input_ids'])}")
print(f"Labels length: {len(sample['labels'])}")
print(f"Input tokens: {tokenizer.convert_ids_to_tokens(sample['input_ids'][:5])}")
print(f"Label tokens: {tokenizer.convert_ids_to_tokens(sample['labels'][:5])}")

## Bước 4: Load Model và Setup LoRA

Tương thích với PyTorch 2.5.1+cu121

In [ ]:
# Load model với PyTorch 2.5.1 compatibility
print(f"📥 Loading model: {MODEL_NAME}")

# Load model với float32 cho stability
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32,  # Sử dụng float32 cho PyTorch 2.5.1
        use_safetensors=True
    )
    print("✅ Model loaded với safetensors")
except:
    print("⚠️ Safetensors failed, trying fallback...")
    model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32
    )
    print("✅ Model loaded với fallback")

print(f"📊 Model có {model.num_parameters():,} parameters")

# Move model to device
model = model.to(device)
print(f"🔧 Model moved to {device}")

# Setup LoRA cho efficient training
print("\n🔧 Setting up LoRA...")
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,                    # LoRA rank (nhỏ hơn để đơn giản)
    lora_alpha=16,          # LoRA alpha
    lora_dropout=0.1,       # LoRA dropout
    target_modules=["q", "v"]  # Target attention modules
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8 if torch.cuda.is_available() else None
)

print("✅ LoRA setup hoàn tất!")

## Bước 5: Setup Training Arguments

Cấu hình tối ưu cho PyTorch 2.5.1 và dataset nhỏ

In [ ]:
# Training arguments đơn giản và hiệu quả
output_dir = "./results/mt5-vi-en-simple"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    
    # Basic training settings
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    
    # Learning rate
    learning_rate=5e-4,
    weight_decay=0.01,
    warmup_steps=50,
    
    # Evaluation
    eval_steps=25,
    save_steps=25,
    logging_steps=10,
    evaluation_strategy="steps",
    
    # Generation settings
    predict_with_generate=True,
    generation_max_length=max_length,
    generation_num_beams=4,
    
    # Tối ưu cho PyTorch 2.5.1
    fp16=False,  # Tắt fp16 cho stability
    dataloader_num_workers=0,
    remove_unused_columns=False,
    
    # Save settings
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Reporting
    report_to=None,  # Tắt wandb
)

print("⚙️ Training arguments:")
print(f"  📁 Output: {output_dir}")
print(f"  🔢 Epochs: {training_args.num_train_epochs}")
print(f"  📏 Batch size: {training_args.per_device_train_batch_size}")
print(f"  📊 Learning rate: {training_args.learning_rate}")
print(f"  🎯 Eval steps: {training_args.eval_steps}")
print(f"  🔥 FP16: {training_args.fp16}")
print("✅ Training arguments sẵn sàng!")

In [ ]:
# Load evaluation metrics
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_preds):
    """Compute BLEU score"""
    predictions, labels = eval_preds
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Decode labels (replace -100 with pad token)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Clean text
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    
    # Compute BLEU
    result = bleu_metric.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]
    )
    
    # Return result
    return {
        "bleu": result["bleu"],
        "prediction_length": np.mean([len(pred.split()) for pred in decoded_preds])
    }

print("📊 Setting up evaluation metrics...")
print("✅ Metrics sẵn sàng - sử dụng BLEU score")

## Bước 6: Training Model

Bắt đầu fine-tuning với LoRA

In [ ]:
# Create trainer
print("🚀 Tạo Trainer...")
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(f"📊 Training dataset: {len(train_dataset)} samples")
print(f"📊 Validation dataset: {len(val_dataset)} samples")

# Test model trước khi training
print("\n🧪 Test model trước training:")
test_input = "translate Vietnamese to English: Xin chào"
inputs = tokenizer.encode(test_input, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(inputs, max_length=50, num_beams=2, do_sample=False)
    before_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Trước training: {before_translation}")

# Bắt đầu training
print("\n🏋️ Bắt đầu training...")
print("💡 Tip: Training sẽ mất vài phút, hãy kiên nhẫn!")

# Start training
trainer.train()

print("\n✅ Training hoàn tất!")

In [ ]:
# Lưu model
print("💾 Lưu model...")
model.save_pretrained(f"{output_dir}/final_model")
tokenizer.save_pretrained(f"{output_dir}/final_model")
print(f"✅ Model đã lưu tại: {output_dir}/final_model")

# Đánh giá cuối cùng
print("\n📊 Đánh giá cuối cùng...")
final_results = trainer.evaluate()
print(f"📈 Final BLEU Score: {final_results.get('eval_bleu', 0):.4f}")
print(f"📉 Final Loss: {final_results.get('eval_loss', 0):.4f}")

# Test model sau training
print("\n🧪 Test model sau training:")
test_input = "translate Vietnamese to English: Xin chào"
inputs = tokenizer.encode(test_input, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(inputs, max_length=50, num_beams=4, do_sample=False)
    after_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Sau training: {after_translation}")

# Vẽ training history nếu có
try:
    import matplotlib.pyplot as plt
    
    # Lấy training logs
    logs = trainer.state.log_history
    
    if logs:
        # Tách train và eval logs
        train_logs = [log for log in logs if 'train_loss' in log]
        eval_logs = [log for log in logs if 'eval_loss' in log]
        
        if train_logs and eval_logs:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
            
            # Training loss
            steps = [log['step'] for log in train_logs]
            losses = [log['train_loss'] for log in train_logs]
            ax1.plot(steps, losses, 'b-', label='Training Loss')
            ax1.set_title('Training Loss')
            ax1.set_xlabel('Steps')
            ax1.set_ylabel('Loss')
            ax1.legend()
            
            # Eval loss
            eval_steps = [log['step'] for log in eval_logs]
            eval_losses = [log['eval_loss'] for log in eval_logs]
            ax2.plot(eval_steps, eval_losses, 'r-', label='Validation Loss')
            ax2.set_title('Validation Loss')
            ax2.set_xlabel('Steps')
            ax2.set_ylabel('Loss')
            ax2.legend()
            
            plt.tight_layout()
            plt.show()
            
            print("📊 Training charts được tạo!")
        else:
            print("⚠️ Không có đủ dữ liệu để vẽ charts")
    else:
        print("⚠️ Không có training logs")
        
except Exception as e:
    print(f"⚠️ Không thể vẽ charts: {e}")

print("\n🎉 Training hoàn tất!")

## Bước 7: Test Translation

Kiểm tra chất lượng dịch thuật với các câu mới

In [ ]:
# Evaluate the fine-tuned model
print("📊 Evaluating fine-tuned model...")
eval_results = trainer.evaluate()

print("\n🎯 Evaluation Results:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# Tạo function dịch thuật đơn giản
def translate_text(vietnamese_text, max_length=64, num_beams=4):
    """Dịch từ tiếng Việt sang tiếng Anh"""
    input_text = f"translate Vietnamese to English: {vietnamese_text}"
    
    # Encode input
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True).to(device)
    
    # Generate translation
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # Decode output
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# Test với validation set
print("🧪 Test trên validation set:")
print("=" * 60)

correct = 0
total = 0

for i in range(len(val_dataset)):
    # Reconstruct original text từ tokenized data
    input_ids = val_dataset[i]['input_ids']
    label_ids = val_dataset[i]['labels']
    
    # Decode để lấy text gốc
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    reference = tokenizer.decode(label_ids, skip_special_tokens=True)
    
    # Extract Vietnamese text từ input
    vietnamese_text = input_text.replace("translate Vietnamese to English: ", "")
    
    # Dịch
    prediction = translate_text(vietnamese_text)
    
    # So sánh (đơn giản)
    if reference.lower().strip() in prediction.lower().strip():
        correct += 1
    total += 1
    
    print(f"🇻🇳 Tiếng Việt: {vietnamese_text}")
    print(f"🎯 Đáp án:     {reference}")
    print(f"🤖 Dự đoán:    {prediction}")
    print(f"{'✅ Đúng' if reference.lower().strip() in prediction.lower().strip() else '❌ Sai'}")
    print("-" * 60)

accuracy = correct / total * 100
print(f"\n📊 Kết quả:")
print(f"Độ chính xác: {accuracy:.1f}% ({correct}/{total})")

if accuracy > 70:
    print("🎉 Tuyệt vời! Model hoạt động tốt!")
elif accuracy > 50:
    print("👍 Khá tốt! Có thể cải thiện thêm.")
else:
    print("⚠️ Cần training thêm hoặc tăng data.")

## Bước 8: Test với câu mới

Thử nghiệm với các câu tiếng Việt chưa có trong training data

In [ ]:
# Create a translation function
def translate_vietnamese(text, max_length=128, num_beams=4):
    """Translate Vietnamese text to English using our fine-tuned model."""
    input_text = f"translate Vietnamese to English: {text}"
    inputs = tokenizer.encode(
        input_text, 
        return_tensors="pt", 
        max_length=max_input_length, 
        truncation=True
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )
    
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# Test với câu mới (không có trong training data)
test_sentences = [
    "Chào bạn",
    "Tôi đói bụng",
    "Bây giờ là mấy giờ?",
    "Tôi cần giúp đỡ",
    "Rất vui được gặp bạn",
    "Hôm nay là thứ mấy?",
    "Tôi không biết",
    "Xin chào mọi người"
]

print("🇻🇳 ➡️ 🇺🇸 TEST TRANSLATION")
print("=" * 50)

for i, vn_text in enumerate(test_sentences, 1):
    translation = translate_vietnamese(vn_text)
    print(f"{i}. 🇻🇳 {vn_text}")
    print(f"   🇺🇸 {translation}")
    print("-" * 50)

# Interactive test - bạn có thể thay đổi câu này
print("\n🎮 TEST INTERACTIVE:")
print("Thay đổi câu dưới đây và chạy lại cell để test:")

your_sentence = "Tôi yêu lập trình"
your_translation = translate_vietnamese(your_sentence)

print(f"\n🇻🇳 Câu của bạn: {your_sentence}")
print(f"🇺🇸 Bản dịch: {your_translation}")

# Tips cho người dùng
print("\n💡 TIPS:")
print("- Model nhỏ nên có thể chưa hoàn hảo")
print("- Câu càng đơn giản thì dịch càng tốt")
print("- Có thể training thêm để cải thiện")
print("- Thử với dataset lớn hơn để có kết quả tốt hơn")

## 🎉 Kết luận

**Chúc mừng! Bạn đã hoàn thành việc fine-tune model dịch Việt-Anh!**

### ✅ Những gì đã làm:
1. **Cài đặt**: PyTorch 2.5.1+cu121 và thư viện cần thiết
2. **Dataset**: Tạo 20 cặp câu Việt-Anh đơn giản
3. **Model**: Load mT5-small và setup LoRA
4. **Training**: Fine-tune với settings tối ưu
5. **Test**: Kiểm tra chất lượng dịch thuật

### 📈 Cách cải thiện:

#### 1. **Dataset lớn hơn**
```python
# Sử dụng dataset từ Hugging Face
from datasets import load_dataset
dataset = load_dataset("Helsinki-NLP/opus-100", "vi-en")
```

#### 2. **Model lớn hơn**
```python
# Thử mT5-base thay vì mT5-small
MODEL_NAME = "google/mt5-base"
```

#### 3. **Training lâu hơn**
```python
# Tăng epochs và giảm learning rate
num_train_epochs=10
learning_rate=3e-4
```

#### 4. **Hyperparameter tuning**
```python
# Thử các settings khác
r=16, lora_alpha=32  # LoRA settings
per_device_train_batch_size=4  # Batch size
```

### 🚀 Bước tiếp theo:

1. **Sử dụng script có sẵn**:
   ```bash
   # Preprocess data
   python -m src.data_processing.preprocess
   
   # Training
   python -m src.training.fine_tune
   
   # Inference
   python -m src.inference.translate --model_path "path/to/model" --text "xin chào"
   ```

2. **Chạy web interface**:
   ```bash
   python main.py
   # Mở http://localhost:8000
   ```

3. **Deploy API**:
   ```bash
   uvicorn main:app --host 0.0.0.0 --port 8000
   ```

### 🎯 Kết quả mong đợi:
- **Small dataset**: BLEU ~0.3-0.5
- **Large dataset**: BLEU ~0.6-0.8
- **Production ready**: BLEU >0.8

### 📚 Tài liệu tham khảo:
- [Hugging Face Transformers](https://huggingface.co/docs/transformers)
- [PEFT Documentation](https://huggingface.co/docs/peft)
- [mT5 Paper](https://arxiv.org/abs/2010.11934)

### 🔧 Troubleshooting:
- **Out of memory**: Giảm batch_size xuống 1
- **Slow training**: Sử dụng GPU hoặc giảm model size
- **Poor quality**: Tăng data hoặc training epochs

**Happy coding! 🎉**